In [1]:
import os
from time import time

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
%matplotlib inline

In [3]:
from skimage.transform import rescale
from skimage import io

In [4]:
import keras
import keras.layers as L
from keras.applications.resnet50 import ResNet50
from keras.models import load_model

Using TensorFlow backend.


In [5]:
import os
from time import time

import numpy as np

from keras.models import load_model

from skimage.transform import rescale
from skimage import io

def adjust_size(img, size=224):
    scaled_img = rescale(img, size/max(img.shape))
    if len(scaled_img.shape) == 2:
        scaled_img = np.stack([scaled_img]*3).transpose(1, 2, 0)
    for i in [0, 1]:
        if scaled_img.shape[i] < size:
            result = np.zeros([size, size, 3])
            gap = (size - scaled_img.shape[i]) // 2
            subslice = [slice(gap, gap + scaled_img.shape[i]), slice(None)]
            result[subslice[i], subslice[1 - i]] = scaled_img
            return result
    return scaled_img

def load_images(path, names, size=224):
    result = np.empty([len(names), size, size, 3])
    for i, name in enumerate(names):
        result[i, :, :, :] = adjust_size(io.imread(os.path.join(path, name)), size=size)
    return result

                                         
def iter_batches(path, target=False, batch_size=100, shuffle=True, cycle=True):
    files = np.array(os.listdir(os.path.join(path, 'images')))
    if target:
        targets = pd.read_csv(os.path.join(path, 'gt.csv'), index_col='filename')
    while True:
        if shuffle:
            np.random.shuffle(files)
        for start in range(0, len(files), batch_size):
            names = files[start: start + batch_size]
            images = load_images(path, names)
            if target is not None:
                yield images, np.array(targets.loc[names]).T[0]
            else:
                yield images
                
        if not cycle: break

def train_classifier(train_gt, train_img_dir, fast_train=False):
    time_begin = time()
    np.random.seed(325)
    names = os.listdir(train_img_dir)
    
    batch_size = 100
    max_time = 10 if fast_train else 20
    
    model = load_model('birds_model.hdf5')
    
    for epoch_id in range(max_epochs):
        np.random.shuffle(names)
        for batch_start in range(0, len(names), batch_size):
            images = load_images(train_img_dir, names[batch_start : batch_start + batch_size])
            targets = [train_gt[name] for name in names[batch_start : batch_start + batch_size]]
            model.train_on_batch(images, targets)
        
        if time() - time_begin > max_time * 60:
            break
    
    model.save('birds_model.hdf5')
    
    time_end = time()
    return time_end - time_begin


def train_classifier(train_gt, train_img_dir, fast_train=False):
    time_begin = time()
    np.random.seed(325)
    names = os.listdir(train_img_dir)
    
    batch_size = 20
    max_time = 5 if fast_train else 20
    max_epochs = 50
    
    model = load_model('birds_model.hdf5')
    
    for epoch_id in range(max_epochs):
        np.random.shuffle(names)
        for batch_id, batch_start in enumerate(range(0, len(names), batch_size)):
            print('epoch_id: {}, batch_id: {}'.format(epoch_id, batch_id))
            images = load_images(train_img_dir, names[batch_start : batch_start + batch_size])
            targets = [train_gt[name] for name in names[batch_start : batch_start + batch_size]]
            model.train_on_batch(images, targets)
        
        if time() - time_begin > max_time * 60:
            break
    
    model.save('birds_model.hdf5')
    
    time_end = time()
    return time_end - time_begin

def classify(model, test_img_dir):
    names = os.listdir(train_img_dir)
    images = load_images(test_img_dir, names)
    predicts = model.predict(images)
    return dict(zip(names, predicts))
    

In [8]:
names = ['{:04d}.jpg'.format(i) for i in range(2500)]
images = load_images('data/00_input/train/images', names)
targets = np.array(pd.read_csv('data/00_input/train/gt.csv', index_col='filename').loc[names]).T[0]

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [9]:
model = load_model('birds_model.hdf5')

In [10]:
predicts = model.predict(images)

In [12]:
np.save('predicts', predicts)

In [20]:
(predicts.argmax(axis=1) == targets).mean()

0.016

In [18]:
predicts.argmax(axis=1)

array([24, 24, 24, ..., 24, 24, 24])

In [6]:
train_classifier(1, 1)

NotADirectoryError: [Errno 20] Not a directory: 1

In [63]:
d = dict(zip(a, b))

In [2]:
def get_model_memory_usage(batch_size, model):
    import numpy as np
    from keras import backend as K

    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    total_memory = 4.0*batch_size*(shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return gbytes

In [3]:
get_model_memory_usage(50, model)

NameError: name 'model' is not defined

In [19]:
imgs = load_images(path, ['{:04d}.jpg'.format(i) for i in [1, 4, 5, 7]])

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [23]:
model.predict(imgs).argmax(axis=1)

array([24, 24, 37,  1])

In [8]:
model = load_model('birds_model.hdf5')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [6]:
# del resnet
# del model

resnet = ResNet50()
output = L.Dense(50, activation='softmax', name='dense')(resnet.get_layer(index=-2).output)
model = keras.models.Model(input=resnet.input, output=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.save('birds_model.hdf5')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  


In [25]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [15]:
path = 'data/00_input/train'
names = ['{:04d}.jpg'.format(i) for i in range(2500)][::50]
images = load_images(path, names)
targets = np.array(pd.read_csv(os.path.join(path, 'gt.csv'), index_col='filename').loc[names]).T[0]

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


NameError: name 'pd' is not defined

In [18]:
model.fit_generator(iter_batches, epochs=10, verbose=2)

Epoch 1/10
 - 57s - loss: 4.8625
Epoch 2/10
 - 42s - loss: 3.5692
Epoch 3/10
 - 42s - loss: 3.8200
Epoch 4/10
 - 44s - loss: 4.0107
Epoch 5/10
 - 43s - loss: 3.6163
Epoch 6/10
 - 42s - loss: 3.3600
Epoch 7/10
 - 44s - loss: 3.1311
Epoch 8/10
 - 43s - loss: 2.9842
Epoch 9/10
 - 46s - loss: 3.2761
Epoch 10/10
 - 45s - loss: 2.5690


In [ ]:
model.fit_generator(iter_batches('data/00_input/train', target=True), steps_per_epoch=10)

Epoch 1/1


/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [313]:
for i in range(2500):
    img = io.imread('data/00_input/train/images/{:04d}.jpg'.format(i))
    if len(img.shape) != 3:
        img = np.stack(img, img, img)
        plt.imshow(img)


TypeError: only size-1 arrays can be converted to Python scalars

In [13]:
!pip3 install pydot

[sudo] password for andrey: 

